# 1  Подгрузка и обзор данных


In [3]:
import pandas as pd

In [4]:
!gdown --id 179rnoPWbcrqCMs7RHf424Ei09gml99WD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=179rnoPWbcrqCMs7RHf424Ei09gml99WD
To: /content/music_project.csv
100% 4.77M/4.77M [00:00<00:00, 29.3MB/s]


In [5]:
df = pd.read_csv('music_project.csv')

In [6]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


* userID — идентификатор пользователя;
* Track — название трека;
* artist — имя исполнителя;
* genre — название жанра;
* City — город пользователя;
* time — время начала прослушивания;
* Day — день недели.
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**
Каждая строка содержит данные о треке и пользователе, из какого он города, в какое время и день недели прослушивал трек.

Замечены 2 проблемы, которые нужно решить:
* названия колонок;
* пропуски в данных.

# 2  Предобработка данных

## 2.1  Стиль заголовков

Приведем названия колонок к единому стилю

In [8]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [9]:
df = df.rename(columns = {
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

## 2.2  Пропуски значений

In [10]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Здесь видим, что для некоторых треков доступна не вся информация. track и artist не влияют на исследование, а вот пропуски в genre могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Но так как у нас нет возможности установить причину пропусков, заменим их на 'unknown'. 


In [11]:
columns_to_replace = ['track', 'artist', 'genre']# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')

In [12]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

## 2.3  Дубликаты

Дубликаты могли появиться при сбое в записи данных. Следует обратить внимание на причины их возникновения. Посчитаем количество явных дубликатов, и удалим их. Убедимся, что дубликаты были удалены.

In [13]:
df.duplicated().sum()

3826

In [14]:
df = df.drop_duplicates().reset_index(drop=True)
df

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
...,...,...,...,...,...,...,...
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61249,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hip,Saint-Petersburg,10:00:00,Monday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [15]:
df.duplicated().sum()

0

Посмотрим на неявные дубликаты

In [16]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [17]:
wrong = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(wrong, name)
df['genre'] = df['genre'].replace('электроника', 'electronic')

In [18]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Заменили одинаковые названия, написанные по-разному, на одно название. Убедились, что замена произошла успешно.

**Выводы**
На предобработке данных привели названия колонок к единому стилю, удалили явные дубликаты и заменили неявные, заменили пропуски на 'unknown' для сохранения данных о жанре.


# 3  Проверка гипотез

## 3.1  Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Для проверки гипотезы необходимо сгруппировать данные по городу и дню недели.

In [19]:
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

In [20]:
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.
Объявим функцию для подсчета прослушанных треков по дням недели и городу, затем объеденим данные в одну таблицу.

In [21]:
def number_tracks(day, city):
    track_list = df[df['day'] == day] 
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [22]:
number_tracks('Monday', 'Moscow')

15740

In [23]:
number_tracks('Monday', 'Saint-Petersburg')

5614

In [24]:
number_tracks('Wednesday', 'Moscow')

11056

In [25]:
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [26]:
number_tracks('Friday', 'Moscow')

15945

In [27]:
number_tracks('Friday', 'Saint-Petersburg')

5895

In [28]:
pd.DataFrame(data=[['Москва', 15740, 5614, 11056], ['Санкт-Петербург', 7003, 15945, 5895]], columns=['city', 'monday', 'wednesday', 'friday'])

,city,monday,wednesday,friday
0,Москва,15740,5614,11056
1,Санкт-Петербург,7003,15945,5895


**Выводы**

Данные показывают разницу поведения пользователей:

* В Москве в среду прослушивают минимальное количество треков, а пик прослушиваний приходится на понедельник и пятницу.
* В Санкт - Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти одинакова и в равной мере уступает среде.

## 3.2  Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. Создадим функцию для подсета популярных жанров по дням недели в разные часы.

In [29]:
moscow_general = df[df['city'] == 'Moscow']

In [30]:
spb_general = df[df['city'] == 'Saint-Petersburg']

In [31]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[(df['day'] == day) & (df['time'] < time2) & (df['time'] > time1)] 
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    return genre_df_sorted[:10]

In [32]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [33]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [34]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [35]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

В понедельник утром в Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.


## 3.3  Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка. 

Посчитаем количество прослушиваний в самых популярных жанрах.

In [36]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
moscow_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [37]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres[:10]

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза.

Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

# 4  Итоги исследования

Проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.